In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine
from pandas_gbq import to_gbq
import pandas_gbq
from airflow import DAG
from datetime import datetime, timedelta
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
def download_data():
    url = "https://drive.google.com/uc?id=1nfktbI7ucHOUwO6EHD2pPHvOf1hN9nmZ"
    destination_file = "videogame.csv"
    response = requests.get(url)

    if response.status_code == 200:
        with open(destination_file, "wb") as file:
            file.write(response.content)
            print(f"The file {destination_file} has been successfully downloaded.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

    df = pd.read_csv(destination_file)
    return df


In [5]:
def transform_data(df):
    dataframes = {}
    dataframe_names = []

    df = df.dropna(axis=1, how='all')
    df = df.drop_duplicates()

    i = 1

    for columna in df.columns:
        if df[columna].isnull().any():
            df_temp = df.dropna(subset=[columna])
            df_temp = df_temp.dropna(axis=1, how='all')
            df_temp = df_temp.rename(columns={columna: 'Plataforma'})

            df_name = f'DF_{i}'
            dataframes[df_name] = df_temp
            dataframe_names.append(df_name)
            i += 1

    dataframes_list = []
    for df_name in dataframe_names:
        df_temp = dataframes[df_name]
        dataframes_list.append(df_temp)

    # Concatena los DataFrames de manera vertical
    df2 = pd.concat(dataframes_list, axis=0)


    df2[['Genero', 'Editorial']] = df2['Genero-Editorial'].str.rsplit('-', n=1, expand=True)
    df2 = df2.drop('Genero-Editorial', axis=1)

    df2['Ventas NA'] = df2['Ventas NA'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas EU'] = df2['Ventas EU'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas JP'] = df2[ 'Ventas JP'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas Otros'] = df2['Ventas Otros'].str.replace(',', '.', regex=True).astype(float)
    df2['Ventas Global'] = df2['Ventas Global'].str.replace(',', '.', regex=True).astype(float)
    return df2

In [16]:
def upload_data(df2, table):

    credentials = service_account.Credentials.from_service_account_file('C:\\Users\\REMO\\Desktop\\Analisis de Datos\\Poyecto Procesamiento\\TDGlobant\\globanttd-a23103587ee1.json')
    # Crea un cliente de BigQuery.
    client = bigquery.Client(credentials=credentials)
    project_id = 'globanttd'
    dataset_id = 'Videogames'
    table_id = table

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_TRUNCATE'  # Opcional: Sobrescribe la tabla existente.

    # Carga el DataFrame en BigQuery.
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job = client.load_table_from_dataframe(df2, table_ref, location='US', job_config=job_config)
    job.result()  # Espera a que se complete el trabajo de carga.

    print(f'Datos cargados en {project_id}.{dataset_id}.{table_id}')

In [7]:
df= download_data()
df.head()

The file videogame.csv has been successfully downloaded.


,Nombre,3DS,PC,PS3,PSP,PSV,X360,PS4,XOne,Año,Genero-Editorial,Unnamed2,Ventas NA,Ventas EU,Ventas JP,Ventas Otros,Ventas Global
0,ZombiU,NaN,NaN,NaN,NaN,NaN,NaN,PS4,NaN,2016,Action-Ubisoft,NaN,0,"0,04","0,01","0,01","0,05"
1,Zombie Army Trilogy,NaN,NaN,NaN,NaN,NaN,NaN,PS4,NaN,2015,Shooter-Rebellion Developments,NaN,"0,04","0,12",0,"0,03","0,2"
2,Zombie Army Trilogy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XOne,2015,Shooter-Rebellion Developments,NaN,"0,04","0,05",0,"0,01","0,1"
3,Zero Escape: Zero Time Dilemma,3DS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,Adventure-Aksys Games,NaN,"0,04",0,"0,01","0,01","0,06"
4,Zero Escape: Zero Time Dilemma,NaN,NaN,NaN,NaN,PSV,NaN,NaN,NaN,2016,Adventure-Aksys Games,NaN,"0,03",0,"0,02","0,01","0,06"


In [8]:
df2= transform_data(df)
df2.head()

,Nombre,Plataforma,Año,Ventas NA,Ventas EU,Ventas JP,Ventas Otros,Ventas Global,Genero,Editorial
3,Zero Escape: Zero Time Dilemma,3DS,2016,0.04,0.0,0.01,0.01,0.06,Adventure,Aksys Games
8,Yowamushi Pedal,3DS,2015,0.00,0.0,0.04,0.00,0.04,Action,Namco Bandai Games
13,Yokai Watch Busters,3DS,2015,0.00,0.0,2.28,0.00,2.28,Action,Level 5
14,Yokai Watch 3,3DS,2016,0.00,0.0,1.27,0.00,1.27,Action,Level 5
15,Yokai Sangokushi,3DS,2016,0.00,0.0,0.55,0.00,0.55,Action,Level 5


In [9]:
unique_years = df2['Año'].unique()

In [33]:

engine = create_engine('sqlite:///:memory:')


df2.to_sql('temp_table', engine, if_exists='replace', index=False)


sql_query1 = "SELECT * FROM temp_table WHERE Año = 2015"
sql_query2 = "SELECT * FROM temp_table WHERE Año = 2016"
sql_query3 = "SELECT * FROM temp_table WHERE Año = 2017"

df_2015 = pd.read_sql(sql_query1, engine)
df_2016 = pd.read_sql(sql_query2, engine)
df_2017 = pd.read_sql(sql_query3, engine)

sql_query4 = "SELECT * FROM temp_table ORDER BY 'Ventas Global' DESC LIMIT 10"
top_10 = pd.read_sql(sql_query, engine)


In [28]:
upload_data(df2,'complete')
upload_data(df_2015,'2015')
upload_data(df_2016,'2016')
upload_data(df_2017,'2017')
upload_data(top_10,'top10')

Datos cargados en globanttd.Videogames.complete
Datos cargados en globanttd.Videogames.2015
Datos cargados en globanttd.Videogames.2016
Datos cargados en globanttd.Videogames.2017
Datos cargados en globanttd.Videogames.top10
